#Adding in city_id & city for each neighborhood (6.16)
###Purpose: 
Add the city_id & city fields to each record in the MongoDB neighborhoods collection.

###Background: 
On my first loop through AirBnB, I didn't save the city_id or city fields from the AirBnBNeighborhood class to MongoDB. I updated the code in my class, but would rather not rescrape everything.

In [2]:
import pandas as pd
from pymongo import MongoClient

db_name = 'airbnb'
coll_name = 'neighborhoods'

client = MongoClient()
db = client[db_name]
coll = db[coll_name]

In [34]:
df = pd.read_csv('../data/neighborhood_list.csv')
df.head()

,neighborhood_id,neighborhood,neighborhood_url,city_id,city
0,0,Alamo Square,/locations/san-francisco/alamo-square,1,san-francisco
1,1,Bayview,/locations/san-francisco/bayview,1,san-francisco
2,2,Bernal Heights,/locations/san-francisco/bernal-heights,1,san-francisco
3,3,Chinatown,/locations/san-francisco/chinatown,1,san-francisco
4,4,Civic Center,/locations/san-francisco/civic-center,1,san-francisco


In [57]:
import airbnb.airbnbneighborhood as airn
reload(airn)
air_hood = airn.AirBnBNeighborhood(db_name = db_name, coll_name = coll_name)
air_hood

In [41]:
for hood in df[['neighborhood_id', 'city_id','city']].to_dict(orient='records'):
    hood_id = hood.pop('neighborhood_id')
    air_hood.pull_from_db(neighborhood_id = hood_id)
    air_hood.add_features(new_features=hood)
    # print air_hood.neighborhood, hood

In [56]:
#confirming
for i in df['neighborhood_id']:
    air_hood.pull_from_db(i)
    print air_hood.d['city']

san-francisco
san-francisco
san-francisco
san-francisco
san-francisco
san-francisco
san-francisco
san-francisco
san-francisco
san-francisco
san-francisco
san-francisco
san-francisco
san-francisco
san-francisco
san-francisco
san-francisco
san-francisco
san-francisco
san-francisco
san-francisco
san-francisco
san-francisco
san-francisco
san-francisco
san-francisco
san-francisco
san-francisco
san-francisco
san-francisco
san-francisco
san-francisco
san-francisco
san-francisco
san-francisco
san-francisco
san-francisco
san-francisco
san-francisco
san-francisco
san-francisco
san-francisco
new-york
new-york
new-york
new-york
new-york
new-york
new-york
new-york
new-york
new-york
new-york
new-york
new-york
new-york
new-york
new-york
new-york
new-york
new-york
new-york
new-york
new-york
new-york
new-york
new-york
new-york
new-york
new-york
new-york
new-york
new-york
new-york
new-york
new-york
new-york
new-york
new-york
new-york
new-york
new-york
new-york
new-york
new-york
new-york
new-york
new-yor

#Creating a mapping table for ammenities (6.16)
###Purpose: 
Create a mapping table from AirBnB's amenity_ids (from the meta data) to the actual amenity.

###Background: 
In the listing, AirBnB provides a list of amenities offered by the listing. However, they don't explicitly state the ammenity. This is meant to create a mapping table from the amenity_id to the amenity.

In [58]:
import airbnb.airbnblisting as airl
from bs4 import BeautifulSoup
import json

In [59]:
reload(airl)
DB_NAME = 'airbnb'
COLL_NAME = 'listings'

air_listing = airl.AirBnBListing(db_name=DB_NAME, coll_name=COLL_NAME)
air_listing

In [60]:
sample_id = '470028'
air_listing.pull_from_db(listing_id = sample_id)
air_listing.url

u'https://www.airbnb.com/rooms/470028'

In [61]:
soup = BeautifulSoup(air_listing.r.content)

In [94]:
list_of_ammenities = []
for x in soup.find('div', {'class':"expandable-content-full"}).find_all('span'):
    list_of_ammenities.append(\
                              {'id':int(x['id'][::-1][:x['id'][::-1].find('-')][::-1]),
                               'amenity':x.get_text().strip()})
df = pd.DataFrame(list_of_ammenities)
df = df.set_index('id').sort()
df.head()
df.to_csv('../data/amenities.csv', index_label='id')

#Adding Neighborhood to listings (6.16)
###Purpose: 
Add the neighborhood information to the listings.


###Background: 
I might have run this code in a notebook. Copying it for reference

In [3]:
import airbnb.airbnblisting as airl

In [4]:
reload(airl)
DB_NAME = 'airbnb'
COLL_NAME = 'listings'

air_listing = airl.AirBnBListing(db_name=DB_NAME, coll_name=COLL_NAME)
air_listing

In [5]:
listing_dict = list(air_listing.coll.find({},{'_id':1}))

In [6]:
for listing in listing_dict:
    listing_id = listing['_id']
    air_listing.pull_from_db(listing_id=listing_id)
    soup = BeautifulSoup(air_listing.r.content)
    try:
        hood = soup.find('div',{'id':'neighborhood-seo-link'}).h3.a.get_text().strip()
        print "Found hood for %s (%s)" % (listing_id, hood)
    except (AttributeError):
        hood = "N/A"
        print "No hood found for %s" % listing_id
    air_listing.add_features(new_features = {'neighborhood':hood})

#Updating the clean_description (6.26)
###Purpose: 
Update the clean descriptions


###Background: 
after some testing, I recreated what the clean decription is, so rerunning across the entire document set

In [29]:
import airbnb.airbnblisting as airl
import pandas as pd

In [30]:
reload(airl)
DB_NAME = 'airbnb'
COLL_NAME = 'listings'

air_listing = airl.AirBnBListing(db_name=DB_NAME, coll_name=COLL_NAME)
air_listing

In [31]:
listing_df = pd.DataFrame(list(air_listing.coll.find({})))

In [32]:
for i in set(listing_df['_id']):
    print "Extracting Features for: %s" % i
    air_listing.pull_from_db(listing_id=i)
    air_listing.extract_and_add_features()

Extracting Features for: 6260181
Extracting Features for: 6643892
Extracting Features for: 5948841
Extracting Features for: 6766738
Extracting Features for: 4104718
Extracting Features for: 3663178
Extracting Features for: 968852
Extracting Features for: 297938
Extracting Features for: 2158401
Extracting Features for: 4163644
Extracting Features for: 6168969
Extracting Features for: 3860632
Extracting Features for: 1696666
Extracting Features for: 4796405
Extracting Features for: 5346515
Extracting Features for: 5347776
Extracting Features for: 6376525
Extracting Features for: 4448920
Extracting Features for: 1246681
Extracting Features for: 52294
Extracting Features for: 3480210
Extracting Features for: 1356545
Extracting Features for: 4784493
Extracting Features for: 4581729
Extracting Features for: 4539325
Extracting Features for: 6654826
Extracting Features for: 1471683
Extracting Features for: 6845554
Extracting Features for: 6515470
Extracting Features for: 5394104
Extracting Fea

In [26]:
air_listing.pull_from_db(listing_id='6751766')

In [34]:
air_listing._clean_description("\nInteraction with Guests\nI'm gone M-F 6AM to 7PM and potentially for the evening depending on events for the day. Happy to give advice or hang out in the evening.\nGetting Around\n2 blocks from a cable car pickup. Bus stops are near and the Bart is 0.7 miles. \nMost places are walkable. \nHave never waited more than two minutes for uber or lyft.\n\n")

'i am gone m f am to pm and potentially for the evening depending on events for the day happy to give advice or hang out in the evening blocks from a cable car pickup bus stops are near and the bart is miles most places are walkable have never waited more than two minutes for uber or lyft'

In [23]:
air_listing.extract_features()

{'address': 'Reservoir Oval East, Bronx (Norwood)',
 'description_clean': u'living room that fits to people comfortably a couch and air bed in a share aptguest have access to kitchen bathroom terrace and also tv and wifi providedthis is a share apt i might be in apt coming in and out but will respect you space the neighborhood is very diversified with different cultures and small stores and also a beautiful park right out front with supermarkets pharmacy and restaurants theres are buses and major subway lines walking distance fro apt i can provide useful information on transportation touristic spot and will answer any questions you may have',
 'description_raw': u'\nThe Space\nLiving room that fits 3 to 5 people comfortably, A couch and air Bed in a share apt\nGuest Access\nguest have access to kitchen ,bathroom ,terrace and also tv and wifi provided\nInteraction with Guests\nthis is a share apt i might be in apt coming in and out but will respect you space.\nThe Neighborhood\nthe neig